### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [5]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    
    df = reviews.sort_values(by=order_by, ascending=True)
    training_df = df.iloc[:training_size, :]
    validation_df = df.iloc[training_size:training_size+testing_size, :] # iloc : inclusive
    

    return training_df, validation_df

In [6]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [8]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [9]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [16]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.482423
2 		 5.906825
3 		 4.133463
4 		 3.099914
5 		 2.421726
6 		 1.942867
7 		 1.587798
8 		 1.315453
9 		 1.101397
10 		 0.930060
11 		 0.790992
12 		 0.676869
13 		 0.582374
14 		 0.503521
15 		 0.437263
16 		 0.381230
17 		 0.333564
18 		 0.292794
19 		 0.257753
20 		 0.227504
21 		 0.201290
22 		 0.178497
23 		 0.158619
24 		 0.141236
25 		 0.126000
26 		 0.112616
27 		 0.100835
28 		 0.090446
29 		 0.081267
30 		 0.073143
31 		 0.065940
32 		 0.059544
33 		 0.053853
34 		 0.048782
35 		 0.044256
36 		 0.040208
37 		 0.036584
38 		 0.033333
39 		 0.030411
40 		 0.027782
41 		 0.025412
42 		 0.023272
43 		 0.021338
44 		 0.019586
45 		 0.017998
46 		 0.016555
47 		 0.015244
48 		 0.014050
49 		 0.012961
50 		 0.011967
51 		 0.011059
52 		 0.010228
53 		 0.009467
54 		 0.008770
55 		 0.008129
56 		 0.007541
57 		 0.007000
58 		 0.006502
59 		 0.006044
60 		 0.005621
61 		 0.005231
62 		 0.004871
63 		 0.004538
64 		 

The mean squared error metrics show a pretty low score after 250 iterations.

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [18]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Get series of indices of users and movies that matches the ordering in training data
    train_user_series = pd.Series(train_data_df.index)
    train_movie_series = pd.Series(train_data_df.columns)
    
    # Find the location of input user_id and movie_id
    user_idx = np.where(train_user_series == user_id)[0][0]
    movie_idx = np.where(train_movie_series == movie_id)[0][0]
    
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[user_idx, :], movie_matrix[:, movie_idx])
   
    return pred

In [31]:
# Take a rating that is a nan
train_data_df.loc[8, 2844]

nan

In [32]:
# Test your function with the first user-movie in the user-movie matrix 
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

6.1928593084379635

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [63]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    
    movie_title = movies.query(f'movie_id == {movie_id}')['movie'].values[0]
    
    print(f'For the user {user_id},\n',
          f'we predict a rating of {prediction:.2f} for <<{movie_title}>>')


In [64]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

For the user 8,
 we predict a rating of 6.19 for <<Fantômas - À l'ombre de la guillotine (1913)>>


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [67]:
val_df.head(1)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
650588,49056,1598822,8,1363308721,2013-03-15 00:52:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [98]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to 
        make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made 
        for each row of val_df from row 0 to num_preds
    '''
    
    val_users = np.array(val_df['user_id'])
    val_movies = np.array(val_df['movie_id'])
    val_ratings = np.array(val_df['rating'])    
    
    print('User\tMovie\tActual\tpredicted')
    
    for i in range(num_preds):
        pred = predict_rating(user_mat, movie_mat, val_users[i], val_movies[i])
        print(f'{val_users[i]:5}\t{val_movies[i]}\t{val_ratings[i]:.2f}\t{pred:.2f}')
        
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

User	Movie	Actual	predicted
49056	1598822	8.00	7.04
49056	289879	9.00	7.63
49056	1563738	9.00	6.80
49056	1458175	4.00	7.77
28599	103639	8.00	6.95
50593	1560985	4.00	3.21


In [99]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

User	Movie	Actual	predicted
49056	1598822	8.00	7.04
49056	289879	9.00	7.63
49056	1563738	9.00	6.80
49056	1458175	4.00	7.77
28599	103639	8.00	6.95
50593	1560985	4.00	3.21


IndexError: index 0 is out of bounds for axis 0 with size 0

### Reason for the IndexError

The 7th movie is a movie that has no ratings. It looks that we are getting new users/movies for test set. Therefore, we are not able to make a prediction for this user-movie pair.